In [5]:
import json
import random
from underthesea import word_tokenize
import sklearn_crfsuite
from sklearn.model_selection import train_test_split
# Bỏ import metrics cũ của sklearn_crfsuite nếu không dùng nữa
# from sklearn_crfsuite import metrics
from collections import defaultdict
import warnings
import re # Thêm re để dùng trong get_category_from_tag nếu cần

# --- Hàm load_data, word2features, prepare_data_with_duplication giữ nguyên ---
# (Copy lại các hàm này từ phiên bản tốt nhất trước đó của bạn)
def load_data(filepath):
    # (Giữ nguyên code hàm load_data)
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
            if "sentences" in data and "sentence" in data["sentences"]:
                 return data["sentences"]["sentence"]
            else:
                 print("Lỗi: Cấu trúc JSON không đúng, không tìm thấy 'sentences.sentence'")
                 if isinstance(data, list):
                      print("Thử giả định file JSON là một danh sách các câu...")
                      return data
                 return []
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file {filepath}")
        return []
    except json.JSONDecodeError:
        print(f"Lỗi: File {filepath} không phải là định dạng JSON hợp lệ.")
        return []
    except Exception as e:
        print(f"Lỗi không xác định khi đọc file: {e}")
        return []

def word2features(sent_tokens, i):
    # (Giữ nguyên code hàm word2features)
    word = sent_tokens[i]
    features = {
        'bias': 1.0, 'word.lower()': word.lower(), 'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(), 'word.isdigit()': word.isdigit(),
        'word.suffix(3)': word[-3:], 'word.prefix(3)': word[:3],
    }
    if i > 0:
        word1 = sent_tokens[i-1]
        features.update({'-1:word.lower()': word1.lower(), '-1:word.istitle()': word1.istitle(), '-1:word.isupper()': word1.isupper()})
    else: features['BOS'] = True
    if i < len(sent_tokens)-1:
        word1 = sent_tokens[i+1]
        features.update({'+1:word.lower()': word1.lower(), '+1:word.istitle()': word1.istitle(), '+1:word.isupper()': word1.isupper()})
    else: features['EOS'] = True
    return features

def prepare_data_with_duplication(sentences_data):
    # (Giữ nguyên code hàm prepare_data_with_duplication)
    final_data = [] # List cuối cùng chứa các cặp (features, labels)
    span_mapping_fail_count = 0
    sentences_with_overlap = 0

    for sentence_idx, sentence_info in enumerate(sentences_data):
        text = sentence_info['text']
        aspects_in = sentence_info.get('aspects', [])

        # Bước 1: Tokenize và tính toán character spans
        tokens = word_tokenize(text)
        if not tokens: continue
        token_spans = []
        current_pos = 0
        for token in tokens:
            try:
                start = text.index(token, current_pos)
                end = start + len(token)
                token_spans.append({'text': token, 'start': start, 'end': end})
                current_pos = end
            except ValueError:
                # print(f"Cảnh báo: Không thể khớp token '{token}' lại với text gốc trong câu {sentence_idx}. Gán span không hợp lệ.") # Giảm bớt log
                token_spans.append({'text': token, 'start': -1, 'end': -1})
                current_pos += len(token)

        # Bước 2: Map aspects to token indices và xác định chồng lấn
        aspect_token_spans = []
        token_to_aspect_indices = defaultdict(list)
        valid_aspects = []

        for aspect_idx, aspect_data in enumerate(aspects_in):
            try:
                from_char = int(aspect_data['from'])
                to_char = int(aspect_data['to'])
                category = aspect_data.get('category')
                term = aspect_data.get('term')

                if category and from_char != -1 and to_char != -1 and from_char < to_char:
                    start_token_idx, end_token_idx = -1, -1
                    for i, span_info in enumerate(token_spans):
                        if span_info['start'] == -1: continue
                        tok_start, tok_end = span_info['start'], span_info['end']
                        if start_token_idx == -1 and tok_start <= from_char < tok_end: start_token_idx = i
                        if tok_start < to_char and tok_start <= (to_char - 1) < tok_end: end_token_idx = i

                    if start_token_idx != -1 and end_token_idx != -1 and start_token_idx <= end_token_idx:
                        current_aspect_info = {'id': aspect_idx,'category': category,'term': term,'start_token': start_token_idx,'end_token': end_token_idx,'involved_in_overlap': False}
                        valid_aspects.append(current_aspect_info)
                        for i in range(start_token_idx, end_token_idx + 1):
                            if i < len(tokens): token_to_aspect_indices[i].append(len(valid_aspects) - 1)
                    else:
                        span_mapping_fail_count += 1
                        # print(f"Cảnh báo: Không thể khớp span ký tự [{from_char}-{to_char}] (term '{term}') với span token câu {sentence_idx}.") # Giảm bớt log

            except (ValueError, TypeError, KeyError): continue

        overlapping_aspect_indices = set()
        for token_idx, mapped_aspect_idxs in token_to_aspect_indices.items():
            if len(mapped_aspect_idxs) > 1:
                for aspect_v_idx in mapped_aspect_idxs:
                    if aspect_v_idx < len(valid_aspects): # Check index validity
                         overlapping_aspect_indices.add(aspect_v_idx)
                         valid_aspects[aspect_v_idx]['involved_in_overlap'] = True

        non_overlapping_aspects = [a for idx, a in enumerate(valid_aspects) if idx not in overlapping_aspect_indices]
        overlapping_aspects = [a for idx, a in enumerate(valid_aspects) if idx in overlapping_aspect_indices]

        # Bước 3: Tạo features
        sentence_features = [word2features(tokens, i) for i in range(len(tokens))]

        # Bước 4: Tạo các mẫu huấn luyện (features, labels)
        generated_samples = []
        if not overlapping_aspects:
            labels = ['O'] * len(tokens)
            processed_non_overlap = set()
            for aspect in non_overlapping_aspects:
                start_tok, end_tok = aspect['start_token'], aspect['end_token']
                category = aspect['category']
                can_add = True
                for i in range(start_tok, end_tok + 1):
                    if i in processed_non_overlap: can_add = False; break
                if can_add:
                    try:
                        labels[start_tok] = f'B-{category}'
                        processed_non_overlap.add(start_tok)
                        for i in range(start_tok + 1, end_tok + 1):
                             if i < len(labels): labels[i] = f'I-{category}'; processed_non_overlap.add(i)
                    except IndexError: pass
            generated_samples.append((sentence_features, labels))
        else:
            sentences_with_overlap += 1
            base_labels = ['O'] * len(tokens)
            processed_non_overlap = set()
            for aspect in non_overlapping_aspects:
                start_tok, end_tok = aspect['start_token'], aspect['end_token']
                category = aspect['category']
                can_add_base = True
                for i in range(start_tok, end_tok + 1):
                     if i in processed_non_overlap: can_add_base = False; break
                if can_add_base:
                    try:
                        base_labels[start_tok] = f'B-{category}'
                        processed_non_overlap.add(start_tok)
                        for i in range(start_tok + 1, end_tok + 1):
                             if i < len(base_labels): base_labels[i] = f'I-{category}'; processed_non_overlap.add(i)
                    except IndexError: pass

            for target_aspect in overlapping_aspects:
                labels_for_target = base_labels[:]
                start_tok, end_tok = target_aspect['start_token'], target_aspect['end_token']
                category = target_aspect['category']
                try:
                    labels_for_target[start_tok] = f'B-{category}'
                    for i in range(start_tok + 1, end_tok + 1):
                        if i < len(labels_for_target): labels_for_target[i] = f'I-{category}'
                    generated_samples.append((sentence_features, labels_for_target))
                except IndexError: pass # Bỏ qua nếu lỗi index

        final_data.extend(generated_samples)

    print("-" * 20)
    print("Thống kê quá trình chuẩn bị dữ liệu (Data Duplication):")
    print(f"- Số câu có aspect chồng lấn được nhân bản (ước lượng): {sentences_with_overlap}") # Đây là số câu gốc có chồng lấn
    print(f"- Số lần không khớp span ký tự với span token: {span_mapping_fail_count}")
    print("-" * 20)
    return final_data


# --- Các hàm mới cho việc đánh giá theo Category ---

def get_category_from_tag(tag):
    """Trích xuất category từ nhãn BIO (ví dụ: 'B-Teaching quality' -> 'Teaching quality')."""
    if tag is None or tag == 'O':
        return None
    # Sử dụng regex để tìm category sau tiền tố B- hoặc I-
    match = re.match(r'^[BI]-(.*)$', tag)
    if match:
        return match.group(1)
    # print(f"Cảnh báo: Không thể trích xuất category từ tag '{tag}'") # Bỏ comment nếu muốn debug tag lạ
    return None # Hoặc trả về tag gốc nếu không có tiền tố B/I?

def bio_tags_to_spans(tags):
    """
    Chuyển đổi một chuỗi nhãn BIO thành một set các cụm (category, start, end).
    Ví dụ: ['O', 'B-CAT1', 'I-CAT1', 'O', 'B-CAT2'] -> {('CAT1', 1, 2), ('CAT2', 4, 4)}
    """
    spans = set()
    current_category = None
    start_index = -1

    for i, tag in enumerate(tags):
        category = get_category_from_tag(tag) # Lấy category (hoặc None)

        if tag.startswith('B-'):
            # Nếu đang có span cũ, lưu lại
            if current_category is not None:
                spans.add((current_category, start_index, i - 1))
            # Bắt đầu span mới
            current_category = category
            start_index = i
        elif tag.startswith('I-'):
            # Nếu không có span trước đó HOẶC category thay đổi -> lỗi I không hợp lệ
            if current_category is None or category != current_category:
                # Đang có I mà không có B hoặc khác category -> kết thúc span cũ (nếu có) và coi I này như O
                if current_category is not None:
                    spans.add((current_category, start_index, i - 1))
                current_category = None
                start_index = -1
            # Else: Vẫn tiếp tục span hiện tại, không cần làm gì (chỉ cập nhật end khi gặp B hoặc O hoặc hết chuỗi)
        else: # Tag là 'O' hoặc không hợp lệ (None)
            # Nếu đang có span, kết thúc và lưu lại
            if current_category is not None:
                spans.add((current_category, start_index, i - 1))
            # Reset
            current_category = None
            start_index = -1

    # Lưu lại span cuối cùng nếu có
    if current_category is not None:
        spans.add((current_category, start_index, len(tags) - 1))

    return spans

def calculate_category_metrics(y_true_bio, y_pred_bio):
    """
    Tính toán Precision, Recall, F1 cho từng category dựa trên so khớp span.

    Args:
        y_true_bio: List các list nhãn BIO thực tế.
        y_pred_bio: List các list nhãn BIO dự đoán.

    Returns:
        dict: Dictionary chứa P, R, F1 cho từng category.
              Ví dụ: {'CAT1': {'P': ..., 'R': ..., 'F1': ...}, ...}
    """
    true_positives = defaultdict(int)
    false_positives = defaultdict(int)
    false_negatives = defaultdict(int)
    all_categories = set()

    if len(y_true_bio) != len(y_pred_bio):
        raise ValueError("Số lượng câu trong y_true và y_pred không khớp!")

    for i in range(len(y_true_bio)):
        true_tags = y_true_bio[i]
        pred_tags = y_pred_bio[i]

        true_spans = bio_tags_to_spans(true_tags)
        pred_spans = bio_tags_to_spans(pred_tags)

        # Cập nhật danh sách các category gặp phải
        for category, _, _ in true_spans: all_categories.add(category)
        for category, _, _ in pred_spans: all_categories.add(category)

        # Tính TP và FP (duyệt qua các span dự đoán)
        for pred_span in pred_spans:
            category = pred_span[0]
            if pred_span in true_spans:
                true_positives[category] += 1
            else:
                false_positives[category] += 1

        # Tính FN (duyệt qua các span thực tế)
        for true_span in true_spans:
            category = true_span[0]
            if true_span not in pred_spans:
                false_negatives[category] += 1

    # Tính toán P, R, F1 cho từng category
    results = {}
    sorted_categories = sorted(list(all_categories)) # Sắp xếp alphabet

    for category in sorted_categories:
        tp = true_positives[category]
        fp = false_positives[category]
        fn = false_negatives[category]

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

        results[category] = {
            'P': precision,
            'R': recall,
            'F1': f1,
            # 'TP': tp, 'FP': fp, 'FN': fn # Bỏ comment nếu muốn xem cả số lượng
        }

    return results

# --- 4. Main Script (Phần đánh giá được cập nhật) ---
input_json_file = 'output_semeval_format_v3.json' # Hoặc file JSON bạn muốn dùng
raw_sentences = load_data(input_json_file)

if not raw_sentences:
    print("Không có dữ liệu để xử lý.")
else:
    print(f"Đã đọc {len(raw_sentences)} câu từ file JSON gốc.")

    prepared_data = prepare_data_with_duplication(raw_sentences)
    if not prepared_data:
         print("Không thể chuẩn bị dữ liệu từ các câu đã đọc.")
    else:
        X = [item[0] for item in prepared_data]
        y = [item[1] for item in prepared_data]
        print(f"Đã chuẩn bị xong {len(X)} mẫu dữ liệu (sau khi nhân bản).")

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        print(f"Kích thước tập huấn luyện: {len(X_train)}")
        print(f"Kích thước tập kiểm thử: {len(X_test)}")

        crf = sklearn_crfsuite.CRF(
            algorithm='lbfgs',
            c1=0.1,
            c2=0.1,
            max_iterations=100,
            all_possible_transitions=True
        )

        print("Bắt đầu huấn luyện mô hình CRF...")
        try:
            crf.fit(X_train, y_train)
            print("Huấn luyện hoàn tất.")

            print("Bắt đầu dự đoán trên tập kiểm thử...")
            y_pred = crf.predict(X_test) # Đây là list các list nhãn BIO dự đoán

            # --- Đánh giá theo Category ---
            print("\n--- Kết quả đánh giá theo Category (Span-based) ---")
            category_results = calculate_category_metrics(y_test, y_pred) # y_test cũng là list các list nhãn BIO

            # In kết quả dạng bảng giống hình ảnh
            print(f"{'Aspect Category':<30} {'Precision':<12} {'Recall':<12} {'F1-score':<12}")
            print("-" * 68)

            all_precisions = []
            all_recalls = []
            all_f1s = []

            for category, metrics_dict in category_results.items():
                p = metrics_dict['P']
                r = metrics_dict['R']
                f1 = metrics_dict['F1']
                all_precisions.append(p)
                all_recalls.append(r)
                all_f1s.append(f1)
                print(f"{category:<30} {p:<12.3%} {r:<12.3%} {f1:<12.3%}") # Format phần trăm

            # --- (Tùy chọn) Tính trung bình Macro ---
            if all_f1s: # Tránh chia cho 0 nếu không có category nào
                 macro_p = sum(all_precisions) / len(all_precisions)
                 macro_r = sum(all_recalls) / len(all_recalls)
                 macro_f1 = sum(all_f1s) / len(all_f1s)
                 # Hoặc tính F1 từ P, R trung bình:
                 # macro_f1_alt = 2 * (macro_p * macro_r) / (macro_p + macro_r) if (macro_p + macro_r) > 0 else 0.0
                 print("-" * 68)
                 print(f"{'Macro Average':<30} {macro_p:<12.3%} {macro_r:<12.3%} {macro_f1:<12.3%}")
            print("-" * 68)


        except Exception as e:
             print(f"Đã xảy ra lỗi trong quá trình huấn luyện hoặc dự đoán/đánh giá: {e}")
             import traceback
             traceback.print_exc()

Đã đọc 15519 câu từ file JSON gốc.
--------------------
Thống kê quá trình chuẩn bị dữ liệu (Data Duplication):
- Số câu có aspect chồng lấn được nhân bản (ước lượng): 1336
- Số lần không khớp span ký tự với span token: 9
--------------------
Đã chuẩn bị xong 17786 mẫu dữ liệu (sau khi nhân bản).
Kích thước tập huấn luyện: 14228
Kích thước tập kiểm thử: 3558
Bắt đầu huấn luyện mô hình CRF...
Huấn luyện hoàn tất.
Bắt đầu dự đoán trên tập kiểm thử...

--- Kết quả đánh giá theo Category (Span-based) ---
Aspect Category                Precision    Recall       F1-score    
--------------------------------------------------------------------
Course information             36.232%      24.752%      29.412%     
General review                 36.774%      19.128%      25.166%     
Learning environment           50.327%      34.222%      40.741%     
Organization and management    23.626%      14.879%      18.259%     
Support from lecturers         27.815%      17.612%      21.568%     
Teach

In [ ]:
import json
import os

def convert_to_semeval_json(input_filepath, output_filepath):
    """
    Chuyển đổi file text chứa dữ liệu ABSA sang định dạng JSON giống SemEval.

    Args:
        input_filepath (str): Đường dẫn đến file text đầu vào.
        output_filepath (str): Đường dẫn để lưu file JSON đầu ra.
    """
    sentences_data = {}

    try:
        with open(input_filepath, 'r', encoding='utf-8') as f:
            # Bỏ qua dòng header
            header = next(f).strip().split('\t')
            print(f"Đã đọc header: {header}") # In header để kiểm tra

            for line_num, line in enumerate(f, 1):
                parts = line.strip().split('\t')

                # Kiểm tra số lượng cột có đủ không
                if len(parts) != 6:
                    print(f"Cảnh báo: Dòng {line_num} không có đủ 6 cột, bỏ qua: {line.strip()}")
                    continue

                review, sentence_component, aspect_text, aspect_category, sentiment_text, sentiment = parts

                # Chuẩn hóa sentiment (ví dụ: Positive -> positive)
                # Nếu sentiment của bạn đã chuẩn rồi thì có thể bỏ qua bước này
                sentiment = sentiment.lower()
                if sentiment not in ["positive", "negative", "neutral"]:
                     # Hoặc xử lý theo cách khác nếu có các giá trị khác
                     print(f"Cảnh báo: Sentiment không xác định '{sentiment}' ở dòng {line_num}, giữ nguyên.")


                # Tìm vị trí 'from' và 'to' của aspect_text trong review
                # Sử dụng review gốc làm text gốc để tìm index
                start_index = review.find(aspect_text)

                if start_index == -1:
                    # Thử tìm trong sentence_component nếu không thấy trong review
                    # Điều này ít khả năng xảy ra nếu cấu trúc file đúng
                    print(f"Cảnh báo: Không tìm thấy aspect_text '{aspect_text}' trong review ở dòng {line_num}. Thử tìm trong sentence_component.")
                    start_index = sentence_component.find(aspect_text)
                    if start_index == -1:
                         print(f"Lỗi: Không tìm thấy aspect_text '{aspect_text}' trong cả review và sentence_component ở dòng {line_num}. Gán from/to = -1.")
                         end_index = -1
                    else:
                         # Nếu tìm thấy trong sentence_component, cần điều chỉnh index dựa trên vị trí của sentence_component trong review
                         # Tuy nhiên, logic này phức tạp và file có vẻ dùng review làm gốc.
                         # Tạm thời vẫn tính 'to' dựa trên len(aspect_text)
                         end_index = start_index + len(aspect_text)
                         print(f"Cảnh báo: aspect_text tìm thấy trong sentence_component, index có thể không chính xác với review gốc.")

                else:
                    end_index = start_index + len(aspect_text)

                # Lưu thông tin aspect
                aspect_info = {
                    # Theo cấu trúc SemEval thường dùng 'term' cho text span
                    "term": aspect_text,
                    "category": aspect_category,
                    "polarity": sentiment,
                    "from": str(start_index), # SemEval thường lưu index dạng string
                    "to": str(end_index)      # SemEval thường lưu index dạng string
                }

                # Nhóm các aspect theo câu (review)
                if review not in sentences_data:
                    sentences_data[review] = {
                        "text": review,
                        # SemEval thường có cả 'aspectTerms' và 'aspectCategories'
                        # Ở đây ta gộp thông tin vào một list 'aspects' cho tiện
                        # vì mỗi entry đã có cả category và term (aspect_text)
                        "aspects": []
                    }
                sentences_data[review]["aspects"].append(aspect_info)

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file đầu vào tại '{input_filepath}'")
        return
    except Exception as e:
        print(f"Đã xảy ra lỗi trong quá trình đọc file: {e}")
        return

    # Chuyển đổi dictionary thành list theo cấu trúc mong muốn
    output_list = list(sentences_data.values())

    # Ghi ra file JSON
    try:
        with open(output_filepath, 'w', encoding='utf-8') as f:
            # ensure_ascii=False để giữ nguyên ký tự tiếng Việt
            # indent=4 để file JSON dễ đọc hơn
            json.dump({"sentences": {"sentence": output_list}}, f, ensure_ascii=False, indent=4)
        print(f"Đã chuyển đổi thành công và lưu vào file: {output_filepath}")
    except Exception as e:
        print(f"Đã xảy ra lỗi trong quá trình ghi file JSON: {e}")

# --- Sử dụng hàm ---
input_file = 'combined_cleaned_file.txt' # Tên file bạn đã tải lên
output_file = 'output_semeval_format.json'

# Kiểm tra xem file input có tồn tại không
if os.path.exists(input_file):
    convert_to_semeval_json(input_file, output_file)
else:
    print(f"Lỗi: File '{input_file}' không tồn tại trong thư mục hiện tại.")